In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/yuewang/Desktop/Desktop_Yue_MacBook_Pro/Nrf2_Project/')
import ccal

In [ ]:
df_exp=pd.read_csv('../../../Downloads/CCLE_expression_22Q2.csv', index_col=0)
df_exp.columns=[i.split(' (')[0] for i in df_exp.columns]
df_tnbc_info=pd.read_csv('./updated_TNBC_list_12032022.csv')
df_exp_tnbc=df_exp[df_exp.index.isin(df_tnbc_info.DepMap_ID)]

In [ ]:
df_523=pd.read_csv('CCLE_TNBC_cell_lines_cellular_state_separating_gene_selection_results_12202022.csv')
df_523.drop('Unnamed: 0', inplace=True, axis=1)
df_523.drop_duplicates('gene', keep='first', inplace=True)
gene_list_523=df_523.gene.unique().tolist()

In [ ]:
df_exp_tnbc_523=df_exp_tnbc[df_exp_tnbc.columns.intersection(gene_list_523+['MYC'])]

In [ ]:
D = df_exp_tnbc_523.T
D1 = ((D - D.mean(axis=0))/D.std(axis=0))
D2  = 100 * ((D1 - D1.min(axis=0))/(D1.max(axis=0) - D1.min(axis=0)))
df_exp_tnbc_523_2 = D2

In [ ]:
#performing NMF
k_nmf= ccal.mf_consensus_cluster_with_ks(
    df_exp_tnbc_523_2,
    list(range(4,10)),
    n_job=4,
    plot_df=False,
    plot_w=True,
    plot_h=True,
    directory_path='/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/'
#     n_clustering=1,
)

In [ ]:
#Performing hierarchical clustering (HC) on H and W matrices
for k in range(4, 10): 
    w=pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/{}/w.tsv'.format(k), sep='\t', index_col=0)

    h=pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/{}/h.tsv'.format(k), sep='\t', index_col=0)

    k_hcc = ccal.hierarchical_consensus_cluster_with_ks(
        w,
        list(range(4,10)),
        n_job = 1,
        distance_function = ccal.compute_correlation_distance,
        n_clustering = 40,
        plot_df = True,
    directory_path='/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/{}/W'.format(k))
    
    k_hcc = ccal.hierarchical_consensus_cluster_with_ks(
        h,
        list(range(4,10)),
        n_job = 1,
        distance_function = ccal.compute_correlation_distance,
        n_clustering = 40,
        plot_df = True,
    directory_path='/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/{}/H'.format(k))

In [ ]:
#This color is consistent with the CCLE state colors in the paper figure
Onco_GPS_colors = [
                '#0000FF',  # F1 EMT 
                '#8B0000',  # F3 EGFR signaling + NFKB signaling
                '#ADD8E6',  # F0 NOTCH signaling + partial-EMT state
                '#E9967A',  # F5 DNA damage response
                '#FF7F24',  # F2 PI3K-Akt + luminal-like state
                '#7FFFD4',  # F6 MAGE protein family
                '#EE3B3B',  # F4
                '#cc99cc',  # 3 Dark magenta  
                '#fce94f',  # 4 Butter

                '#fe2c54',  # 6 Redish pink 
                '#AFF8DB',  # 7 Green
                '#ffad01',  # 8 Orange yellow

                '#AAAAFF',  # 10 Periwinkle

                '#FA8072',  # 12 Salmon
                '#ffccf9',  # 13 Pink 
                '#90EE90',  # 14 Light green 
                '#c4a000',  # 15
                '#e6194b',  # 16
                '#3cb44b',  # 17
                '#ffe119']  # 18

In [ ]:
#To generate CCLE TNBC cell line identity map
w=pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/for_map_generation/CCLE_model_w_edited.tsv', sep='\t', index_col=0)
h=pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/for_map_generation/CCLE_model_h_edited.tsv', sep='\t', index_col=0)
wT =w.T
wT.index.name = h.index.name

h_clustering = pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/for_map_generation/CCLE_model_HC_cluster_x_column_edited.tsv', sep='\t', index_col=0)
h_element_labels = h_clustering.apply(lambda column: column.argmax())#.str.lstrip('C').astype(int)
h_element_labels.value_counts()

        
gps_map = ccal.GPSMap(
                w = wT,
                h = h,
                w_n_pull = 3,
                w_pull_power = 3,
                h_n_pull = 3,
                h_pull_power = 3,
                plot = False,
                mds_random_seed = 27)

gps_map.set_element_labels(
                        w_or_h = 'h',
                        element_labels = h_element_labels,
                        n_grid = 64,
                        label_colors = Onco_GPS_colors[0:7],
                        bandwidth_factor = 6)


gps_map.plot_gps_map(
                w_or_h = 'h',
                element_marker_size = 24,
                title = 
                   'CCLE TNBC Model State Map',
                html_file_path = 
                   '/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/maps/CCLE_TNBC_model_cellular_state_map_11132024.html')     